In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.request import Request, urlopen

In [111]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV,\
RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
url = 'https://www.footballdb.com/players/patrick-mahomes-mahompa01/gamelogs/2022'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}) # Make a get request to retrieve the page
webpage = urlopen(req).read()

In [3]:
soup = BeautifulSoup(webpage, 'html.parser') # Pass the page contents to beautiful soup for parsing
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="initial-scale=1.0, width=device-width" name="viewport"/>
<title>Patrick Mahomes 2022 Gamelogs | The Football Database</title>
<meta content="Patrick Mahomes gamelogs for the 2022 season." name="description"/>
<meta content="index,follow" name="robots"/>
<meta content="FootballDB.com" property="og:site_name"/>
<meta content="Patrick Mahomes 2022 Gamelogs | The Football Database" property="og:title">
<meta content="website" property="og:type"/>
<meta content="Patrick Mahomes gamelogs for the 2022 season." property="og:description"/>
<meta content="https://www.footballdb.com/images/footballdb_200x200.png" property="og:image"/>
<meta content="https://www.footballdb.com/players/patrick-mahomes-mahompa01/gamelogs/2022" property="og:url"/>
<meta content="summary" name="twitter:card"/>
<meta content="@TheFootballDB" name="twitter:site"/>
<meta content="https://

In [82]:
# 6 tables, could create a function to extract the data in each table
test1 = soup.find_all('table')
len(test1)

6

In [27]:
test = soup.find('table',{'class':'statistics scrollable'})

In [32]:
test

<table class="statistics scrollable" data-fixed-columns="1">
<thead>
<tr class="header center"><th> </th><th> </th><th> </th><th> </th><th colspan="15">Passing</th><th> </th></tr>
<tr class="header right"><th class="center">Date</th><th class="center" style="padding-left:10px;">Team</th><th class="center">Opp</th><th class="center">Lg</th><th>Att</th><th>Cmp</th><th>Pct</th><th>Yds</th><th>YPA</th><th>TD</th><th><span class="hidden-xs">TD%</span><span class="visible-xs-inline">T%</span></th><th>Int</th><th><span class="hidden-xs">Int%</span><span class="visible-xs-inline">I%</span></th><th>Lg</th><th>FD</th><th>20+</th><th>Sack</th><th>Loss</th><th>Rate</th><th class="center">Res</th></tr>
</thead>
<tbody>
<tr class="row0 preseason right"><td class="center nowrap"><a href="/games/boxscore/kansas-city-chiefs-vs-chicago-bears-2022081304" title="Kansas City Chiefs @ Chicago Bears Box Score - 08/13/22">08/13/22</a></td><td class="center">KC</td><td class="center nowrap">@ CHI</td><td class

In [41]:
row_headers = []
for x in test.find_all('tr'):
    for y in x.find_all('th'):
        row_headers.append(y.text)
row_headers = row_headers[6:]

In [46]:
table_stats = []
for x in test.find_all('tr'):
    td_tags = x.find_all('td')
    if td_tags == []:
        continue
    else:
        pass
    td_val = [y.text for y in td_tags]
    table_stats.append(td_val)
table_stats

[['08/13/22',
  'KC',
  '@ CHI',
  'NFL',
  '7',
  '6',
  '85.7',
  '60',
  '8.6',
  '1',
  '14.3',
  '0',
  '0.0',
  '19',
  '4',
  '0',
  '0',
  '0',
  '142.0',
  'L, 19-14'],
 ['08/20/22',
  'KC',
  'vs WAS',
  'NFL',
  '19',
  '12',
  '63.2',
  '162',
  '8.5',
  '2',
  '10.5',
  '0',
  '0.0',
  '39',
  '9',
  '2',
  '0',
  '0',
  '125.3',
  'W, 24-14'],
 ['08/25/22',
  'KC',
  'vs GB',
  'NFL',
  '0',
  '0',
  '0.0',
  '0',
  '0.0',
  '0',
  '0.0',
  '0',
  '0.0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0.0',
  'W, 17-10'],
 ['09/11/22',
  'KC',
  '@ ARI',
  'NFL',
  '39',
  '30',
  '76.9',
  '360',
  '9.2',
  '5',
  '12.8',
  '0',
  '0.0',
  '35',
  '22',
  '5',
  '0',
  '0',
  '144.2',
  'W, 44-21'],
 ['09/15/22',
  'KC',
  'vs LAC',
  'NFL',
  '35',
  '24',
  '68.6',
  '235',
  '6.7',
  '2',
  '5.7',
  '0',
  '0.0',
  '41t',
  '11',
  '3',
  '1',
  '9',
  '106.2',
  'W, 27-24'],
 ['09/25/22',
  'KC',
  '@ IND',
  'NFL',
  '35',
  '20',
  '57.1',
  '262',
  '7.5',
  '1',
  '2.9',
 

In [71]:
mahomes = pd.DataFrame(table_stats,columns=row_headers)

In [72]:
mahomes

,Date,Team,Opp,Lg,Att,Cmp,Pct,Yds,YPA,TD,TD%T%,Int,Int%I%,Lg,FD,20+,Sack,Loss,Rate,Res
0,08/13/22,KC,@ CHI,NFL,7,6,85.7,60,8.6,1,14.3,0,0.0,19,4,0,0,0,142.0,"L, 19-14"
1,08/20/22,KC,vs WAS,NFL,19,12,63.2,162,8.5,2,10.5,0,0.0,39,9,2,0,0,125.3,"W, 24-14"
2,08/25/22,KC,vs GB,NFL,0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0.0,"W, 17-10"
3,09/11/22,KC,@ ARI,NFL,39,30,76.9,360,9.2,5,12.8,0,0.0,35,22,5,0,0,144.2,"W, 44-21"
4,09/15/22,KC,vs LAC,NFL,35,24,68.6,235,6.7,2,5.7,0,0.0,41t,11,3,1,9,106.2,"W, 27-24"
5,09/25/22,KC,@ IND,NFL,35,20,57.1,262,7.5,1,2.9,1,2.9,53,12,5,1,5,78.5,"L, 20-17"
6,10/02/22,KC,@ TB,NFL,37,23,62.2,249,6.7,3,8.1,1,2.7,36,17,3,3,21,97.7,"W, 41-31"
7,10/10/22,KC,vs LV,NFL,43,29,67.4,292,6.8,4,9.3,0,0.0,36,18,3,3,27,117.6,"W, 30-29"
8,10/16/22,KC,vs BUF,NFL,40,25,62.5,338,8.4,2,5.0,2,5.0,42t,16,6,3,19,85.2,"L, 24-20"
9,10/23/22,KC,@ SF,NFL,34,25,73.5,423,12.4,3,8.8,1,2.9,57,19,7,1,6,132.4,"W, 44-23"


In [73]:
# Manual testing
mahomes.drop([23,24,25])

,Date,Team,Opp,Lg,Att,Cmp,Pct,Yds,YPA,TD,TD%T%,Int,Int%I%,Lg,FD,20+,Sack,Loss,Rate,Res
0,08/13/22,KC,@ CHI,NFL,7,6,85.7,60,8.6,1,14.3,0,0.0,19,4,0,0,0,142.0,"L, 19-14"
1,08/20/22,KC,vs WAS,NFL,19,12,63.2,162,8.5,2,10.5,0,0.0,39,9,2,0,0,125.3,"W, 24-14"
2,08/25/22,KC,vs GB,NFL,0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,0,0,0,0.0,"W, 17-10"
3,09/11/22,KC,@ ARI,NFL,39,30,76.9,360,9.2,5,12.8,0,0.0,35,22,5,0,0,144.2,"W, 44-21"
4,09/15/22,KC,vs LAC,NFL,35,24,68.6,235,6.7,2,5.7,0,0.0,41t,11,3,1,9,106.2,"W, 27-24"
5,09/25/22,KC,@ IND,NFL,35,20,57.1,262,7.5,1,2.9,1,2.9,53,12,5,1,5,78.5,"L, 20-17"
6,10/02/22,KC,@ TB,NFL,37,23,62.2,249,6.7,3,8.1,1,2.7,36,17,3,3,21,97.7,"W, 41-31"
7,10/10/22,KC,vs LV,NFL,43,29,67.4,292,6.8,4,9.3,0,0.0,36,18,3,3,27,117.6,"W, 30-29"
8,10/16/22,KC,vs BUF,NFL,40,25,62.5,338,8.4,2,5.0,2,5.0,42t,16,6,3,19,85.2,"L, 24-20"
9,10/23/22,KC,@ SF,NFL,34,25,73.5,423,12.4,3,8.8,1,2.9,57,19,7,1,6,132.4,"W, 44-23"


In [74]:
#Manual cleaning, will run again with oneHot encoding Opponent team
mahomes = mahomes[['Yds','TD','Int']]
mahomes

,Yds,TD,Int
0,60,1,0
1,162,2,0
2,0,0,0
3,360,5,0
4,235,2,0
5,262,1,1
6,249,3,1
7,292,4,0
8,338,2,2
9,423,3,1


In [75]:
mahomes = mahomes.drop(index=[0,1,2,20,21,22,23,24,25])

In [78]:
mahomes = mahomes.reset_index().drop(columns=['index'])

In [88]:
mahomes = mahomes.astype(int)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Yds     17 non-null     int64
 1   TD      17 non-null     int64
 2   Int     17 non-null     int64
dtypes: int64(3)
memory usage: 536.0 bytes


In [99]:
# Fantasy Pts = FP
"""
Passing Yards: 1 point per 25 yards
Passing Touchdowns: 4 points
Passing Interceptions: -2 points
Rushing Yards: 1 point per 10 yards
Rushing Touchdowns: 6 points
Receptions: 1 points (only if using PPR scoring)
Receiving Yards: 1 point per 10 yards
Receiving Touchdowns: 6 points
2-Point Conversions: 2 points
Fumbles Lost: -2 points
Fumble Recovered for a Touchdown: 6 points
"""
mahomes['FP'] = (mahomes['Yds']/25) + (mahomes['TD']*4) - (mahomes['Int']*2)

In [100]:
mahomes

,Yds,TD,Int,FP
0,360,5,0,34.40
1,235,2,0,17.40
2,262,1,1,12.48
3,249,3,1,19.96
4,292,4,0,27.68
5,338,2,2,17.52
6,423,3,1,26.92
7,446,1,1,19.84
8,331,4,1,27.24
9,329,3,0,25.16


In [103]:
X = mahomes.drop(columns='FP')
y = mahomes['FP']

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [107]:
model = LinearRegression()

In [108]:
model.fit(X_train,y_train)

LinearRegression()

In [109]:
predictions = model.predict(X_test)

In [114]:
predictions

array([16.96, 25.16, 17.4 , 14.8 , 19.96])

In [112]:
print('MSE: ', mean_squared_error(y_test, predictions))

MSE:  7.573064690121713e-30


In [113]:
print('MAE: ', mean_absolute_error(y_test, predictions))

MAE:  2.1316282072803005e-15


In [116]:
error = pd.DataFrame(y[-5:])

In [117]:
error['y_pred'] = predictions

In [124]:
error['error'] = abs(error['FP'] - error['y_pred'])

In [126]:
error['%'] = error['error']/error['FP']

In [127]:
error

,FP,y_pred,error,%
12,20.08,16.96,3.12,0.155378
13,21.44,25.16,3.72,0.173507
14,16.96,17.40,0.44,0.025943
15,23.12,14.80,8.32,0.359862
16,12.08,19.96,7.88,0.652318
